## Notes: 
line of thinking
documented motivation
understand what er are doing


proprocess 3 subjects and provide/document the cleaning times / bad channels, document the bad components

- Preprocessing Filtering, re-referencening, ICA
- Data cleaning: Time, channel and subjects
- ERP peak analysis Extract the study-relevant ERP peak subjectwise (e.g. one value per subject) and statistically test them. RQ: On which ERP-peaks do we find major difference between the conditions?
- Time Frequency analysis Calculate an induced time-frequency analysis of the main experimental contrast RQ: What oscillations underley our effect of interest?
- either:	- Mass Univariate Use a multiple regression of the main experimental contrast, controlling for reaction time (you need to calculate RT yourself). RQ: When/Where do we find differences between our conditions? Is there an influence of reaction time?
		OR
		- Decoding analysis Decode the main contrast of the experiment across time RQ: When is information about the conditions in our data available?

## Dataset Summary and Exploration
P300: A visual oddball experiment with a prolonged effect at 300ms.

In [1]:
"""
import libraries
"""
import mne
import ccs_eeg_utils
import mne  # read raw
from mne_bids import BIDSPath, read_raw_bids
import matplotlib.pyplot as plt
import matplotlib
import os  # make dir
from config import *

from matplotlib.pyplot import figure

matplotlib.use("Agg")
#%matplotlib inline

import numpy as np

In [2]:
def band_pass_filtering(raw, subject_id):
    """
    applys the firwin banpassfilter
    """
    print("filtering ...")

    # plot raw data before band-pass filtering
    raw.plot_psd(
        area_mode="range",
        tmax=10.0,
        average=False,
        xscale="linear",
    )
    ccs_eeg_utils.save_plot("01freq_before_filtering", bids_path)

    # apply filter
    raw_f = raw.copy().filter(l_freq, h_freq, fir_design=fir_design)
    raw_f.plot_psd(area_mode="range", tmax=10.0, average=False, xscale="log")
    ccs_eeg_utils.save_plot("02freq_after_filtering", bids_path)

    raw.set_montage("standard_1020", match_case=False)

    return raw_f

In [3]:
def plot_each_channel(subject_id):
    """
    plot data
    """
    print("plotting each channel ...")

    # create dir to save plots if not existent
    if not os.path.exists(str(str(bids_path)[:-37] + "results/03whole_overlay/")):
        os.makedirs(str(str(bids_path)[:-37] + "results/03whole_overlay/"))

    # Extract a single channel and plot the whole timeseries.
    for channel_number in range(1, len(raw.ch_names)):  # 1

        # show filtered -> low frequencies
        fig = plt.figure()
        a1 = fig.add_axes([0, 0, 1, 1])
        a1.plot((raw[channel_number - 1][0].T))
        a1.plot((raw_f[channel_number - 1][0].T))
        a1.legend(["raw", "band-pass filtered"])
        plt.title("Subject: " + str(subject_id) + ", channel: " + str(channel_number))
        plt.ylabel("potential [µV]")
        plt.xlabel("sample")
        # a1.set_xlim(0,25)
        # a1.set_ylim(-0.00003,0.00003)
        plt.savefig(
            str(
                str(bids_path)[:-37]
                + "results/03whole_overlay/channel"
                + str(channel_number)
                + ".png"
            ),
            bbox_inches="tight",
        )
        # plt.show()
        plt.close(fig)  # don't display figure

    # show raw -> high frequnecies
    channel_number = 1
    fig = plt.figure()
    a1 = fig.add_axes([0, 0, 1, 1])
    a1.plot((raw[channel_number - 1][0].T))
    a1.plot((raw_f[channel_number - 1][0].T))
    a1.legend(["raw", "band-pass filtered"])
    plt.title("Subject: " + str(subject_id) + ", channel: " + str(channel_number))
    plt.ylabel("potential [µV]")
    plt.xlabel("sample")
    a1.set_xlim(0, 25)
    a1.set_ylim(0.0199, 0.01993)
    ccs_eeg_utils.save_plot("04zoom_raw", bids_path)
    plt.close(fig)  # don't display figure

    # show filtered -> high frequnecies are filtered
    fig = plt.figure()
    a1 = fig.add_axes([0, 0, 1, 1])
    a1.plot((raw[channel_number - 1][0].T))
    a1.plot((raw_f[channel_number - 1][0].T))
    a1.legend(["raw", "band-pass filtered"])
    plt.title("Subject: " + str(subject_id) + ", channel: " + str(channel_number))
    plt.ylabel("potential [µV]")
    plt.xlabel("sample")
    a1.set_xlim(0, 25)
    a1.set_ylim(-0.00003, 0.00003)
    ccs_eeg_utils.save_plot("05zoom_filtered", bids_path)
    plt.close(fig)  # don't display figure

In [4]:
def cleaning(raw_f):
    # clean manually
    #%matplotlib qt
    raw_f.plot(n_channels=len(raw_f.ch_names))

    # get bad annotations
    bad_ix = [i for i, a in enumerate(raw_f.annotations) if a["description"] == "BAD_"]

    # save bad annotations
    raw_f.annotations[bad_ix].save(
        str(str(bids_path)[:-37] + "results/badannotations.csv")
    )

    # read bad annotations
    # annotations = mne.read_annotations(str(str(bids_path)[:-37] + "results/badannotations.csv"))

    # append bad annotations
    # raw_f.annotations.append(annotations.onset,annotations.duration,annotations.description)  # duplicates

    # remove abd channels
    raw_f.info["bads"] = []

    # intepolate bad channels
    # raw_f.interpolate_bads()  # no bad channel found

    # save cleaned data
    raw.save(
        str(str(bids_path)[:-37] + "results/after_cleaning_raw.fif"), overwrite=True
    )

    # load cleaned data
    # raw_cleaned = mne.io.read_raw_fif(str(str(bids_path)[:-37] + "results/after_cleaning_raw.fif"))

    """TODO raw vs raw_f"""

    # visualize effect of cleaning on ERP
    evts, evts_dict = mne.events_from_annotations(raw)
    wanted_keys = [e for e in evts_dict.keys() if "stimulus" in e]
    evts_dict_stim = dict((k, evts_dict[k]) for k in wanted_keys if k in evts_dict)

    # get epochs with and without rejection
    epochs = mne.Epochs(
        raw, evts, evts_dict_stim, tmin=-0.1, tmax=1, reject_by_annotation=False
    )
    epochs_manual = mne.Epochs(
        raw, evts, evts_dict_stim, tmin=-0.1, tmax=1, reject_by_annotation=True
    )
    reject_criteria = dict(
        eeg=200e-6, eog=200e-6  # 100 µV # HAD TO INCREASE IT HERE, 100 was too harsh
    )  # 200 µV
    epochs_thresh = mne.Epochs(
        raw,
        evts,
        evts_dict_stim,
        tmin=-0.1,
        tmax=1,
        reject=reject_criteria,
        reject_by_annotation=False,
    )

    # from matplotlib import pyplot as plt
    # compare
    # plt.plot([0,:])
    mne.viz.plot_compare_evokeds(
        {
            "raw": epochs.average(),
            "clean": epochs_manual.average(),
            "thresh": epochs_thresh.average(),
        },
        picks="Cz",
    )

In [5]:
def ICA(raw_f_f):
    """
    ICA
    """
    print("running ICA ...")
    raw_f_f.filter(l_freq_ica, h_freq_ica, fir_design=fir_design_ica)

    # Generate an ICA object
    ica = mne.preprocessing.ICA(
        n_components=0.9999999, random_state=93, method=ica_method, max_iter=200
    )

    # fit the ICA on the data
    ica.fit(raw_f, verbose=True)

    # plot components
    ica.plot_components(picks=range(ica.n_components_ - 1))
    ccs_eeg_utils.save_plot("06ICA_components", bids_path)

    # plot properties
    # create dir to save plots if not existent

    if not os.path.exists(str(str(bids_path)[:-37] + "results/07ICA_properties/")):
        os.makedirs(str(str(bids_path)[:-37] + "results/07ICA_properties/"))
    for component in range(ica.n_components_):
        ica.plot_properties(
            raw_f, picks=component, psd_args={"fmax": 35.0}, reject=None
        )
        # save plot
        plt.savefig(
            str(
                str(bids_path)[:-37]
                + "results/07ICA_properties/component"
                + str(component)
                + ".png"
            ),
            bbox_inches="tight",
        )

        # don't display figure
        plt.close()

    """ TODO generalize for all subjects"""
    """
    # Artefacts of subject one
    heartbeat_artefacts = []
    blink_artefacts = [3]
    eye_artefacts = [0,1,2,6,24,29]
    muscle_artefacts = [5,7,10,13,16,18,21,23,28,29] # often non-stationary activity, small ERPs, "square-root spectrum", one electrode?, Yaw, below/behind ear, lowwe back of the head, 
    noisy_electrodes = [] # single active electrode, small ERP, strong 50/60hz line noise
    other_artefacts = []
    # brain components ideally show strong and clear ERPs, activity stationary over whole experiment, powerspectrum including alpha/beta peak at ~10hz, 
    """
    # Artefacts of subject two
    heartbeat_artefacts = []
    blink_artefacts = [0]
    eye_artefacts = []
    muscle_artefacts = [
        12,
        19,
        25,
    ]  # often non-stationary activity, small ERPs, "square-root spectrum", one electrode?, Yaw, below/behind ear, lowwe back of the head,
    noisy_electrodes = [
        29
    ]  # single active electrode, small ERP, strong 50/60hz line noise
    other_artefacts = [3, 7, 8, 22, 23]
    # brain components ideally show strong and clear ERPs, activity stationary over whole experiment, powerspectrum including alpha/beta peak at ~10hz,

    exclude = []
    exclude.extend(heartbeat_artefacts)
    exclude.extend(blink_artefacts)
    exclude.extend(eye_artefacts)
    exclude.extend(muscle_artefacts)
    exclude.extend(other_artefacts)
    print("excluded components:", exclude)
    ica.exclude = exclude

    # add events
    evts, evts_dict_stim = add_events(raw_f)

    # before ICA
    epochs = mne.Epochs(
        ica.get_sources(raw_f), evts, evts_dict_stim, tmin=-0.2, tmax=0.8
    )
    epochs.average(picks=picked_channel_num).plot()  # 13 for Pz?
    ccs_eeg_utils.save_plot("08" + str(picked_channel) + "_before_ICA", bids_path)

    # after ICA
    epochs = mne.Epochs(raw_f, evts, evts_dict_stim, tmin=epoch_tmin, tmax=epoch_tmax)
    epochs.average().plot(picks=picked_channel)
    ccs_eeg_utils.save_plot("09" + str(picked_channel) + "_after_ICA", bids_path)

    # before / after overlay
    reconst_raw_f = raw_f.copy()
    ica.apply(reconst_raw_f, exclude=exclude)

    # raw_f.plot()
    # reconst_raw_f.plot()
    ica.plot_overlay(raw_f, exclude=exclude)
    ccs_eeg_utils.save_plot("10before_after_overlay", bids_path)

    # save ICA
    ica.save(str(str(bids_path)[:-4] + "_chrei-ica.fif"))

In [6]:
def add_events(raw):
    """
    add events
    """
    print("adding events ...")
    evts, evts_dict = mne.events_from_annotations(raw_f)

    # get all keys which contain "stimulus"
    wanted_keys = [
        e for e in evts_dict.keys() if "stimulus" in e
    ]  # subset the large event-dictionairy

    evts_dict_stim = dict((k, evts_dict[k]) for k in wanted_keys if k in evts_dict)

    return evts, evts_dict_stim

In [7]:
def epoching():
    # Choosing Pz for P3 because this is the site most widely used in the literature and
    # recommended by Kappenman et al.
    # Cut the raw_f data to one channel using `raw_f.pick_channels(["Pz"])` -

    evts, evts_dict_stim = add_events(raw_f)

    raw_f.pick_channels([picked_channel])
    print(raw_f)
    plt.plot(raw_f[:, :][0].T)
    ccs_eeg_utils.save_plot("11" + str(picked_channel), bids_path)

    raw_f.info

    # These values reflect the values in the bids `*_events.tsv` file.
    raw_f.annotations

    # Epoch the data
    epochs = mne.Epochs(
        raw_f, evts, evts_dict_stim, tmin=epoch_tmin, tmax=epoch_tmax, baseline=baseline
    )  # tmin and tmax chosen as recommended by Kappenman et al.

    # Plot all trials
    plt.plot(np.squeeze(epochs.get_data()[:, 0, :].T))
    ccs_eeg_utils.save_plot("12trials", bids_path)

    return epochs, evts, evts_dict_stim

In [8]:
def get_peak_value(data, timestamp):
    """
    get peak values from timestamp
    """
    peak_time_index = data.time_as_index(timestamp)
    data_frame = data.to_data_frame(picks=picked_channel)
    peak_value = data_frame.at[peak_time_index[0], picked_channel]

    return peak_value

In [9]:
def create_peak_data(target_peak, distractor_peak):
    """
    create array where first column is the data and second column is the label
    where: target -> 1, distractor -> 0

    peak value | label
    -----------|------
    0.1234     | 1
    0.2345     | 0
    """

    # ini matrix with zeros
    erp_data = np.zeros(((last_subject) * 2, 2))

    # fill targets
    for row in range(last_subject):
        erp_data[row][0] = target_peak[row]  # values
        erp_data[row][1] = 1  # labels

    # fill distractors
    for row in range(last_subject):
        erp_data[row + last_subject][0] = distractor_peak[row]
        # erp_data[row+last_subject][1] = 0 # already 0s

    print("erp_data:", erp_data)
    print("erp_data.shape:", erp_data.shape)

    return erp_data

In [10]:
def erp_analysis():
    # indexing and averaging epochs
    target = epochs[["stimulus:{}{}".format(k, k) for k in [1, 2, 3, 4, 5]]].average()
    distractor = epochs[
        [
            "stimulus:{}{}".format(k, j)
            for k in [1, 2, 3, 4, 5]
            for j in [1, 2, 3, 4, 5]
            if k != j
        ]
    ].average()

    # plotting
    mne.viz.plot_compare_evokeds([target, distractor])
    ccs_eeg_utils.save_plot("13epochs_average", bids_path)

    """
    get peaks
    """
    # get peak times target
    _, peak_time_target = target.get_peak(mode="pos")  # get peak time
    print("peak_time_target:", peak_time_target)
    peak_time_target_list.append(peak_time_target)

    # get peak value target
    peak_value_target = get_peak_value(target, peak_time_target)
    print("peak_value_target:", peak_value_target)
    peak_value_target_list.append(peak_value_target)  # from epoch to evoked

    # get peak times distractor
    _, peak_time_distractor = distractor.get_peak(mode="pos")
    print("peak_time_distractor:", peak_time_distractor)
    peak_time_distractor_list.append(peak_time_distractor)

    # get peak value distractor
    peak_value_distractor = get_peak_value(distractor, peak_time_distractor)
    print("peak_value_distractor:", peak_value_distractor)
    peak_value_distractor_list.append(peak_value_distractor)

    # print(target_peak_t,distractor_peak_t)

    return (
        peak_time_target_list,
        peak_value_target_list,
        peak_time_distractor_list,
        peak_value_distractor_list,
    )

In [11]:
def erp_differences(
    peak_target_list,
    peak_distractor_list,
    title="title",
    xlabel="xlabel",
    ylabel="# subjects",
):
    peak_difference_list = np.subtract(peak_target_list, peak_distractor_list)
    #%matplotlib inline
    _ = plt.hist(
        peak_difference_list, bins="auto"
    )  # arguments are passed to np.histogram
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

    average_difference = np.average(peak_difference_list)
    meadian_difference = np.median(peak_difference_list)
    std_dev_difference = np.std(peak_difference_list)

    # print(average_difference, meadian_difference, std_dev_difference)
    return average_difference, meadian_difference, std_dev_difference

In [12]:
def permutation_test(peak_target, peak_distractor):
    # create one data array with peak values and labels
    peak_data = create_peak_data(peak_target, peak_distractor)

    # perform permutation test
    t_obs, p_values, H0 = mne.stats.permutation_t_test(peak_data, verbose=True)
    # print('t_obs:', t_obs)
    # print('p_values:', p_values)
    # print('H0:', H0)

    return t_obs, p_values

In [13]:
"""
pipeline

load data
read annotations
resample to 256 Hz
re-reference to P9 and P10
(Extract a single channel and plot the whole timeseries)
"""
# path where to save the datasets.
bids_root = "../local/bids"
last_subject

# time of peak
peak_time_target_list = []
peak_time_distractor_list = []
peak_value_target_list = []
peak_value_distractor_list = []

for subject in range(frist_subject, last_subject + 1):
    print("subject =", subject)
    subject_id = f"{subject:03}"

    bids_path = BIDSPath(
        subject=subject_id,
        task="P3",
        session="P3",
        datatype="eeg",
        suffix="eeg",
        root=bids_root,
    )

    # read the file
    raw = read_raw_bids(bids_path)

    # fix the annotations readin
    ccs_eeg_utils.read_annotations_core(bids_path, raw)

    # load data
    raw.load_data()

    # delay shift
    # todo

    # resample / downsample frequency
    raw.resample(
        sfreq, npad="auto"
    )  # set sampling frequency to 256 Hz as in Kappenman et al.

    # rerefernecing to P9 and P10 becuase Kappenman et al. says
    # "we find that P9 and P10 provide cleaner signals than the traditional mastoid sites"
    raw.set_eeg_reference(ref_channels=ref_channels, verbose=None)

    # add channel locations
    raw.set_montage("standard_1020", match_case=False)

    # band pass filtering between 0.1 and 54 hz
    raw_f = band_pass_filtering(raw, subject_id)

    # plot each channel
    # plot_each_channel(subject_id) # takes a while

    # remove large muscle artifacts, extreme voltage offsets, or break periods
    # cleaning(raw_f)

    # ICA
    # ICA(raw_f)

    epochs, evts, evts_dict_stim = epoching()

    (
        peak_time_target_list,
        peak_value_target_list,
        peak_time_distractor_list,
        peak_value_distractor_list,
    ) = erp_analysis()

# histograms
(
    average_time_difference,
    meadian_time_difference,
    std_dev_time_difference,
) = erp_differences(
    peak_time_target_list,
    peak_time_distractor_list,
    title="Delay of target ERP relative to distractor ERP",
    xlabel="relative delay in ms",
)
print(
    "average_time_difference: ",
    average_time_difference,
    "\nmeadian_time_difference:",
    meadian_time_difference,
    "\n std_dev_time_difference:",
    std_dev_time_difference,
)

(
    average_value_difference,
    meadian_value_difference,
    std_dev_value_difference,
) = erp_differences(
    peak_value_target_list,
    peak_value_distractor_list,
    title="Delay of target ERP relative to distractor ERP",
    xlabel="relative delay in ms",
)
print(
    "average_value_difference: ",
    average_value_difference,
    "\nmeadian_value_difference:",
    meadian_value_difference,
    "\n std_dev_value_difference:",
    std_dev_value_difference,
)

# permutation tests
t_obs_time, p_values_time = permutation_test(
    peak_time_target_list, peak_time_distractor_list
)
t_obs_value, p_values_value = permutation_test(
    peak_value_target_list, peak_value_distractor_list
)

subject = 1
Reading ../local/bids/sub-001/ses-P3/eeg/sub-001_ses-P3_task-P3_eeg.fdt
Reading events from ../local/bids/sub-001/ses-P3/eeg/sub-001_ses-P3_task-P3_events.tsv.
Reading channel info from ../local/bids/sub-001/ses-P3/eeg/sub-001_ses-P3_task-P3_channels.tsv.
Reading 0 ... 478207  =      0.000 ...   466.999 secs...


<ipython-input-13-269fec25a671>:34: RuntimeWarning: Data file name in EEG.data (sub-001_task-P3_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-001_ses-P3_task-P3_eeg.fdt).
  raw = read_raw_bids(bids_path)
<ipython-input-13-269fec25a671>:34: RuntimeWarning: Did not find any coordsystem.json associated with sub-001_ses-P3_task-P3.

The search_str was "../local/bids/sub-001/**/sub-001_ses-P3*coordsystem.json"
  raw = read_raw_bids(bids_path)


EEG channel type selected for re-referencing
Applying a custom EEG reference.
filtering ...
Effective window size : 8.000 (s)
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.4 - 54 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.40
- Lower transition bandwidth: 0.40 Hz (-6 dB cutoff frequency: 0.20 Hz)
- Upper passband edge: 54.00 Hz
- Upper transition bandwidth: 13.50 Hz (-6 dB cutoff frequency: 60.75 Hz)
- Filter length: 2113 samples (8.254 sec)

Effective window size : 8.000 (s)
adding events ...
Used Annotations descriptions: ['response:201', 'response:202', 'stimulus:11', 'stimulus:12', 'stimulus:13', 'stimulus:14', 'stimulus:15', 'stimulus:21', 'stimulus:22', 'stimulus:23', 'stimulus:24', 'stimulus:25', 'stimulus:31', 'stimulus:32', 'stimulus:33

<ipython-input-13-269fec25a671>:34: RuntimeWarning: Did not find any coordsystem.json associated with sub-002_ses-P3_task-P3.

The search_str was "../local/bids/sub-002/**/sub-002_ses-P3*coordsystem.json"
  raw = read_raw_bids(bids_path)


EEG channel type selected for re-referencing
Applying a custom EEG reference.
filtering ...
Effective window size : 8.000 (s)
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.4 - 54 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.40
- Lower transition bandwidth: 0.40 Hz (-6 dB cutoff frequency: 0.20 Hz)
- Upper passband edge: 54.00 Hz
- Upper transition bandwidth: 13.50 Hz (-6 dB cutoff frequency: 60.75 Hz)
- Filter length: 2113 samples (8.254 sec)

Effective window size : 8.000 (s)
adding events ...
Used Annotations descriptions: ['response:201', 'response:202', 'stimulus:11', 'stimulus:12', 'stimulus:13', 'stimulus:14', 'stimulus:15', 'stimulus:21', 'stimulus:22', 'stimulus:23', 'stimulus:24', 'stimulus:25', 'stimulus:31', 'stimulus:32', 'stimulus:33

<ipython-input-13-269fec25a671>:34: RuntimeWarning: Data file name in EEG.data (sub-003_task-P3_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-003_ses-P3_task-P3_eeg.fdt).
  raw = read_raw_bids(bids_path)
<ipython-input-13-269fec25a671>:34: RuntimeWarning: Did not find any coordsystem.json associated with sub-003_ses-P3_task-P3.

The search_str was "../local/bids/sub-003/**/sub-003_ses-P3*coordsystem.json"
  raw = read_raw_bids(bids_path)


EEG channel type selected for re-referencing
Applying a custom EEG reference.
filtering ...
Effective window size : 8.000 (s)
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.4 - 54 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.40
- Lower transition bandwidth: 0.40 Hz (-6 dB cutoff frequency: 0.20 Hz)
- Upper passband edge: 54.00 Hz
- Upper transition bandwidth: 13.50 Hz (-6 dB cutoff frequency: 60.75 Hz)
- Filter length: 2113 samples (8.254 sec)

Effective window size : 8.000 (s)
adding events ...
Used Annotations descriptions: ['response:201', 'response:202', 'stimulus:11', 'stimulus:12', 'stimulus:13', 'stimulus:14', 'stimulus:15', 'stimulus:21', 'stimulus:22', 'stimulus:23', 'stimulus:24', 'stimulus:25', 'stimulus:31', 'stimulus:32', 'stimulus:33

<ipython-input-11-1a38f678f62e>:16: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [14]:
raw.info

<Info | 11 non-empty values
 bads: []
 ch_names: FP1, F3, F7, FC3, C3, C5, P3, P7, P9, PO7, PO3, O1, Oz, Pz, CPz, ...
 chs: 30 EEG, 3 EOG
 custom_ref_applied: True
 dig: 33 items (3 Cardinal, 30 EEG)
 highpass: 0.0 Hz
 line_freq: 60
 lowpass: 128.0 Hz
 meas_date: unspecified
 nchan: 33
 projs: []
 sfreq: 256.0 Hz
 subject_info: 4 items (dict)
>

In [15]:
raw_f.info

<Info | 11 non-empty values
 bads: []
 ch_names: Pz
 chs: 1 EEG
 custom_ref_applied: True
 dig: 33 items (3 Cardinal, 30 EEG)
 highpass: 0.4 Hz
 line_freq: 60
 lowpass: 54.0 Hz
 meas_date: unspecified
 nchan: 1
 projs: []
 sfreq: 256.0 Hz
 subject_info: 4 items (dict)
>

In [16]:
epochs, evts, evts_dict_stim = epoching()

adding events ...
Used Annotations descriptions: ['response:201', 'response:202', 'stimulus:11', 'stimulus:12', 'stimulus:13', 'stimulus:14', 'stimulus:15', 'stimulus:21', 'stimulus:22', 'stimulus:23', 'stimulus:24', 'stimulus:25', 'stimulus:31', 'stimulus:32', 'stimulus:33', 'stimulus:34', 'stimulus:35', 'stimulus:41', 'stimulus:42', 'stimulus:43', 'stimulus:44', 'stimulus:45', 'stimulus:51', 'stimulus:52', 'stimulus:53', 'stimulus:54', 'stimulus:55']
<RawEEGLAB | sub-003_ses-P3_task-P3_eeg.fdt, 1 x 96512 (377.0 s), ~785 kB, data loaded>
Not setting metadata
Not setting metadata
200 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 200 events and 257 original time points ...
0 bad epochs dropped


In [17]:
"""# ensure all conditions have the same counts, as the ANOVA expects a fully balanced data matrix and 
# does not forgive imbalances that generously (risk of type-I error).
epochs.equalize_event_counts(evts_dict_stim) # TODO not sure if it's good that 100 epoches are droped

# Factor to down-sample the temporal dimension of the TFR computed by
# tfr_morlet.
decim = 2
frequencies = np.arange(7, 30, 3)  # define frequencies of interest
n_cycles = frequencies / frequencies[0]
zero_mean = False  # don't correct morlet wavelet to be of mean zero
# To have a true wavelet zero_mean should be True but here for illustration
# purposes it helps to spot the evoked response."""

"# ensure all conditions have the same counts, as the ANOVA expects a fully balanced data matrix and \n# does not forgive imbalances that generously (risk of type-I error).\nepochs.equalize_event_counts(evts_dict_stim) # TODO not sure if it's good that 100 epoches are droped\n\n# Factor to down-sample the temporal dimension of the TFR computed by\n# tfr_morlet.\ndecim = 2\nfrequencies = np.arange(7, 30, 3)  # define frequencies of interest\nn_cycles = frequencies / frequencies[0]\nzero_mean = False  # don't correct morlet wavelet to be of mean zero\n# To have a true wavelet zero_mean should be True but here for illustration\n# purposes it helps to spot the evoked response."

In [18]:
"""from mne.time_frequency import tfr_morlet"""

'from mne.time_frequency import tfr_morlet'

In [19]:
"""import numpy as np
import matplotlib.pyplot as plt

import mne
from mne.time_frequency import tfr_morlet
from mne.stats import f_threshold_mway_rm, f_mway_rm, fdr_correction
from mne.datasets import sample"""

'import numpy as np\nimport matplotlib.pyplot as plt\n\nimport mne\nfrom mne.time_frequency import tfr_morlet\nfrom mne.stats import f_threshold_mway_rm, f_mway_rm, fdr_correction\nfrom mne.datasets import sample'

In [20]:
"""# Create TFR (time-frequency representations) for all conditions
epochs_power = list()
for condition in [epochs[k] for k in evts_dict_stim]:
    this_tfr = tfr_morlet(condition, frequencies, n_cycles=n_cycles,
                          decim=decim, average=False, zero_mean=zero_mean,
                          return_itc=False)
    this_tfr.apply_baseline(mode='ratio', baseline=(None, 0))
    this_power = this_tfr.data[:, 0, :, :]  # we only have one channel.
    epochs_power.append(this_power)"""

"# Create TFR (time-frequency representations) for all conditions\nepochs_power = list()\nfor condition in [epochs[k] for k in evts_dict_stim]:\n    this_tfr = tfr_morlet(condition, frequencies, n_cycles=n_cycles,\n                          decim=decim, average=False, zero_mean=zero_mean,\n                          return_itc=False)\n    this_tfr.apply_baseline(mode='ratio', baseline=(None, 0))\n    this_power = this_tfr.data[:, 0, :, :]  # we only have one channel.\n    epochs_power.append(this_power)"

In [21]:
"""# Setup repeated measures ANOVA
# We will tell the ANOVA how to interpret the data matrix in terms of factors. 
# This is done via the factor levels argument which is a list of the number factor levels for each factor.
n_conditions = len(epochs.event_id)
n_replications = int(epochs.events.shape[0] / n_conditions)

factor_levels = [2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]  # number of levels in each factor
effects = 'A*B'  # this is the default signature for computing all effects
# Other possible options are 'A' or 'B' for the corresponding main effects
# or 'A:B' for the interaction effect only (this notation is borrowed from the
# R formula language)
n_frequencies = len(frequencies)
times = 1e3 * epochs.times[::decim]
n_times = len(times)"""

"# Setup repeated measures ANOVA\n# We will tell the ANOVA how to interpret the data matrix in terms of factors. \n# This is done via the factor levels argument which is a list of the number factor levels for each factor.\nn_conditions = len(epochs.event_id)\nn_replications = int(epochs.events.shape[0] / n_conditions)\n\nfactor_levels = [2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]  # number of levels in each factor\neffects = 'A*B'  # this is the default signature for computing all effects\n# Other possible options are 'A' or 'B' for the corresponding main effects\n# or 'A:B' for the interaction effect only (this notation is borrowed from the\n# R formula language)\nn_frequencies = len(frequencies)\ntimes = 1e3 * epochs.times[::decim]\nn_times = len(times)"

In [22]:
"""# Now we’ll assemble the data matrix and swap axes so the trial replications are the first dimension and the conditions are the second dimension.

data = np.swapaxes(np.asarray(epochs_power), 1, 0)
# reshape last two dimensions in one mass-univariate observation-vector
data = data.reshape(n_replications, n_conditions, n_frequencies * n_times)

# so we have replications * conditions * observations:
print(data.shape)
print(n_conditions)"""

'# Now we’ll assemble the data matrix and swap axes so the trial replications are the first dimension and the conditions are the second dimension.\n\ndata = np.swapaxes(np.asarray(epochs_power), 1, 0)\n# reshape last two dimensions in one mass-univariate observation-vector\ndata = data.reshape(n_replications, n_conditions, n_frequencies * n_times)\n\n# so we have replications * conditions * observations:\nprint(data.shape)\nprint(n_conditions)'

In [23]:
"""# Now we’re ready to run our repeated measures ANOVA.

# Note. As we treat trials as subjects, the test only accounts for time locked responses,
# despite the ‘induced’ approach. For analysis for induced power at the group level averaged TRFs are required.
print('f_mway_rm')
fvals, pvals = f_mway_rm(data, factor_levels, effects=effects)

effect_labels = ['modality', 'location', 'modality by location']

# let's visualize our effects by computing f-images
for effect, sig, effect_label in zip(fvals, pvals, effect_labels):
    print('for')
    plt.figure()
    # show naive F-values in gray
    plt.imshow(effect.reshape(8, 211), cmap=plt.cm.gray, extent=[times[0],
               times[-1], frequencies[0], frequencies[-1]], aspect='auto',
               origin='lower')
    # create mask for significant Time-frequency locations
    effect = np.ma.masked_array(effect, [sig > .05])
    plt.imshow(effect.reshape(8, 211), cmap='RdBu_r', extent=[times[0],
               times[-1], frequencies[0], frequencies[-1]], aspect='auto',
               origin='lower')
    plt.colorbar()
    plt.xlabel('Time (ms)')
    plt.ylabel('Frequency (Hz)')
    plt.title(r"Time-locked response for '%s' (%s)" % (effect_label, ch_name))
    plt.show()"""

'# Now we’re ready to run our repeated measures ANOVA.\n\n# Note. As we treat trials as subjects, the test only accounts for time locked responses,\n# despite the ‘induced’ approach. For analysis for induced power at the group level averaged TRFs are required.\nprint(\'f_mway_rm\')\nfvals, pvals = f_mway_rm(data, factor_levels, effects=effects)\n\neffect_labels = [\'modality\', \'location\', \'modality by location\']\n\n# let\'s visualize our effects by computing f-images\nfor effect, sig, effect_label in zip(fvals, pvals, effect_labels):\n    print(\'for\')\n    plt.figure()\n    # show naive F-values in gray\n    plt.imshow(effect.reshape(8, 211), cmap=plt.cm.gray, extent=[times[0],\n               times[-1], frequencies[0], frequencies[-1]], aspect=\'auto\',\n               origin=\'lower\')\n    # create mask for significant Time-frequency locations\n    effect = np.ma.masked_array(effect, [sig > .05])\n    plt.imshow(effect.reshape(8, 211), cmap=\'RdBu_r\', extent=[times[0],\n    

## Cleaning Data

In [24]:
"""
exercise 3 but is it necessary?
"""

'\nexercise 3 but is it necessary?\n'